In [1]:
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時的例外工具
from selenium.common.exceptions import TimeoutException

# 等待某個元素的出現
from selenium.webdriver.support.ui import WebDriverWait

# 期待元素出現並執行下一個指令
from selenium.webdriver.support import expected_conditions as EC

# 透過什麼方式選取元素
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 強制等待
from time import sleep

# 反爬蟲
import undetected_chromedriver as uc

# 執行 command 的時候用的
import os

# 整理 json 使用的工具
import json

#計算程式執行的時間
import time

# 平行任務處理
from concurrent.futures import ThreadPoolExecutor as tpe

from fake_useragent import UserAgent 
ua = UserAgent(cache= True)

my_options = uc.ChromeOptions()

my_options.add_argument("--start-maximized") # 視窗最大化
my_options.add_argument("--incognito") # 無痕
my_options.add_argument("--disable-popup-blocking") # 禁用彈出攔截
my_options.add_argument("--disable-notifiactions") # 關閉推波通知
my_options.add_argument("--lang=zh-TW") # 設定為繁體中文
my_options.add_argument("--headless")
# my_options.add_argument("--user-agent="+ua.random)

folderPath = "ZhongzhengRestaurant"
if os.path.exists(folderPath) != True:    
    os.makedirs(folderPath)
    


In [2]:
def page_link(link):
   
    # 自動取得Chrom driver 不用再額外取得 executable_path="chromedriver.exe"
    driver = uc.Chrome(options=my_options)
    
    # GoogleMap網址
    url = "https://www.google.com.tw/maps/@23.546162,120.6402133,8z?hl=zh-TW"
    
    driver.get(url)
    
    sleep(1)
    
    # 計算每跑一次跑多久
    start = time.time()
        
    # 輸入關鍵字搜尋
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).send_keys(link)
    
    # 按下搜尋按鈕
    driver.find_element(
        By.CSS_SELECTOR,
        "button#searchbox-searchbutton"
    ).click()
        
    sleep (3)
        
    # 找搜尋元素結果的div
    focus = driver.find_elements(
        By.CSS_SELECTOR,
        "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
    )[1]

    offset = 0
    innerHeight = 0
    count = 0  # 累計無效滾動次數
    limit = 5  # 最大無效滾動次數
    Flag = True
    counter = 0
        
    # 開始滾輪找到最底部
    while Flag:
            
        try:
            # 等待底部元素出現
            WebDriverWait(driver, 0.1).until(
                EC.presence_of_element_located( 
                    (By.CSS_SELECTOR, "span.HlvSq") 
                )
            )
            
            break
            
        except TimeoutException:

           # offset: 拉槓到頁面頂端的距離
            offset = driver.execute_script('return arguments[0].scrollTop', focus)

            # 執行js指令捲動頁面
            driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', focus)
            
            # innerHeight: 頁面高度 = 拉槓到頁面頂端的距離
            innerHeight = driver.execute_script('return arguments[0].scrollHeight = arguments[0].scrollTop', focus)
            
            # 如果「拉槓到頁面頂端的距離」(offset)等於「頁面高度 = 拉槓到頁面頂端的距離」(innerHeight)，代表搜尋可能逾時
            if offset == innerHeight:
                count += 1                
              
            # 計數器等於限制數則搜尋確實逾時了,需重新整理並搜尋
            if count == limit:
                
                counter+=1
                
                print(f"{link}需重新整理,搜尋逾時,本次刷新第{counter}次")
                
                # 網頁重新整理
                driver.execute_script('window.location.replace(window.location.href)')
                
                # 重新搜尋
                driver.find_element(
                    By.CSS_SELECTOR,
                    "button#searchbox-searchbutton"
                 ).click()
                
                sleep(3)

                # 找搜尋結果的div
                focus = driver.find_elements(
                    By.CSS_SELECTOR,
                    "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
                    )[1]

                # 重新定義需要的變數
                offset = 0
                innerHeight = 0
                count = 0  # 累計無效滾動次數
                
            sleep(2)
            
    #尋找所有元素的href    
    href_ = driver.find_elements(
        By.CSS_SELECTOR,
        "div[data-js-log-root] a[aria-label]"
    )
   
    # 放網址的lict
    listData=[]
    counter = 0    
    for href in href_:
        listData.append({
        "href":href.get_attribute("href")
        
    })
        counter+=1
        
    # 把餐廳網址存成json檔        
    with open(f"{folderPath}/ZhongzhengRestaurant.json","a",encoding="utf-8") as f:
        f.write(json.dumps(listData, ensure_ascii=False))
    
    # 清除搜尋的關鍵字
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).clear()   
        
    end = time.time()
    print(f'{link},總共有:{counter}筆,已完成\n')
    print("process_time 測量時間：%f 秒\n" % (end - start))
        
    driver.quit()


In [3]:
def multiprocess():
    
    links = ["中正區三元街餐廳","中正區丹陽街餐廳","中正區仁愛路１段餐廳","中正區仁愛路２段餐廳","中正區信義路１段餐廳","中正區信義路２段餐廳","中正區信陽街餐廳","中正區八德路１段餐廳","中正區公園路餐廳","中正區凱達格蘭大道餐廳","中正區北平東路餐廳","中正區北平西路餐廳","中正區南昌路１段餐廳","中正區南昌路２段餐廳","中正區南海路餐廳","中正區南陽街餐廳","中正區博愛路餐廳","中正區同安街餐廳","中正區和平西路１段餐廳","中正區和平西路２段餐廳","中正區大埔街餐廳","中正區天津街餐廳","中正區寧波東街餐廳","中正區寧波西街餐廳","中正區寶慶路餐廳","中正區市民大道１段餐廳","中正區市民大道２段餐廳","中正區市民大道３段餐廳","中正區師大路餐廳","中正區常德街餐廳","中正區廈門街餐廳","中正區廣州街餐廳","中正區延平南路餐廳","中正區徐州路餐廳","中正區忠孝東路１段餐廳","中正區忠孝東路２段餐廳","中正區忠孝西路１段餐廳","中正區思源街餐廳","中正區惠安街餐廳","中正區愛國東路餐廳","中正區愛國西路餐廳","中正區懷寧街餐廳","中正區新生南路１段餐廳","中正區晉江街餐廳","中正區杭州北路餐廳","中正區杭州南路１段餐廳","中正區杭州南路２段餐廳","中正區林森北路餐廳","中正區林森南路餐廳","中正區桃源街餐廳","中正區武昌街１段餐廳","中正區水源路餐廳","中正區永春街餐廳","中正區永綏街餐廳","中正區汀州路１段餐廳","中正區汀州路２段餐廳","中正區汀州路３段餐廳","中正區沅陵街餐廳","中正區泉州街餐廳","中正區泰安街餐廳","中正區湖口街餐廳","中正區漢口街１段餐廳","中正區潮州街餐廳","中正區濟南路１段餐廳","中正區濟南路２段餐廳","中正區牯嶺街餐廳","中正區福州街餐廳","中正區秀山街餐廳","中正區紹興北街餐廳","中正區紹興南街餐廳","中正區羅斯福路１段餐廳","中正區羅斯福路２段餐廳","中正區羅斯福路３段餐廳","中正區羅斯福路４段餐廳","中正區臨沂街餐廳","中正區自強市場第三棟餐廳","中正區莒光路餐廳","中正區衡陽路餐廳","中正區襄陽路餐廳","中正區西藏路餐廳","中正區許昌街餐廳","中正區詔安街餐廳","中正區貴陽街１段餐廳","中正區辛亥路１段餐廳","中正區連雲街餐廳","中正區酉陽街餐廳","中正區重慶南路１段餐廳","中正區重慶南路２段餐廳","中正區重慶南路３段餐廳","中正區金山北路餐廳","中正區金山南路１段餐廳","中正區金華街餐廳","中正區金門街餐廳","中正區銅山街餐廳","中正區鎮江街餐廳","中正區長沙街１段餐廳","中正區開封街１段餐廳","中正區青島東路餐廳","中正區青島西路餐廳","中正區館前路餐廳","中正區齊東街餐廳"]
    
    # 平行運算x2,
    with tpe(max_workers=2) as executor:
        executor.map(page_link,links) # 執行page_link涵式,links涵式需要的變數

In [4]:
if __name__=="__main__":
    multiprocess()

中正區三元街餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區丹陽街餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區三元街餐廳,總共有:120筆,已完成

process_time 測量時間：112.354476 秒

中正區丹陽街餐廳,總共有:120筆,已完成

process_time 測量時間：123.192507 秒

中正區信義路２段餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區信義路１段餐廳,總共有:120筆,已完成

process_time 測量時間：72.187714 秒

中正區信陽街餐廳,總共有:120筆,已完成

process_time 測量時間：63.674710 秒

中正區八德路１段餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區公園路餐廳,總共有:120筆,已完成

process_time 測量時間：66.135028 秒

中正區凱達格蘭大道餐廳,總共有:120筆,已完成

process_time 測量時間：65.625958 秒

中正區北平東路餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區北平西路餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區北平東路餐廳,總共有:120筆,已完成

process_time 測量時間：130.839713 秒

中正區南昌路１段餐廳,總共有:120筆,已完成

process_time 測量時間：63.608717 秒

中正區南陽街餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區博愛路餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區南陽街餐廳,總共有:120筆,已完成

process_time 測量時間：125.974850 秒

中正區同安街餐廳,總共有:120筆,已完成

process_time 測量時間：73.620842 秒

中正區和平西路１段餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區和平西路１段餐廳,總共有:20筆,已完成

process_time 測量時間：57.709685 秒

中正區和平西路２段餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區大埔街餐廳,總共有:12筆,已完成

process_time 測量時間：3.231824 秒

中正區寧波東街餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區天津街餐廳需重新整理,搜尋逾時,本次刷新第1次
中正區

In [8]:
import json

folderPath = "ZhongzhengRestaurant"
with open(f"{folderPath}/ZhongzhengRestaurant.json", "r", encoding='utf-8') as file:
    
    # 取得 json 字串
    strJson = file.read()
    
    # 用,分割
    aList = strJson.split(",")
    
    a = len(aList)
    print(a)
    
    b = len(set(aList))
    print(b)
    
    filter_ = list(set(aList))
    
    # 把重複的網址篩選掉,並存回json檔
    with open(f"{folderPath}/ZhongzhengRestaurantFilter.json","a",encoding="utf-8") as f:
        f.write(json.dumps(filter_, ensure_ascii=False))
    
    

8827
3520
